In [1]:
pip install scgraph>=1.0.0

Note: you may need to restart the kernel to use updated packages.


In [2]:
from scgraph.geographs.marnet import marnet_geograph

# Get the shortest path between 
output = marnet_geograph.get_shortest_path(
    origin_node={"latitude": 31.23,"longitude": 121.47}, 
    destination_node={"latitude": 32.08,"longitude": -81.09}
)
print('Length: ',output['length']) #=> Length:  19596.4653
print(str([[i['latitude'],i['longitude']] for i in output['coordinate_path']]))

Length:  19596.4653
[[31.23, 121.47], [30.7324, 121.8274], [30.8345, 122.6776], [31.3, 122.9], [33.5458, 126.5397], [34.2, 127.6], [35.0, 129.2], [38.2106, 134.0843], [40.0, 137.2662], [41.2, 139.4], [41.9, 141.25], [41.9, 143.3], [43.2, 146.0], [46.0975, 153.3524], [46.1961, 153.6877], [47.4449, 157.9333], [48.1099, 160.1942], [48.3248, 160.925], [48.4755, 161.4373], [50.0, 169.0188], [50.1542, 169.7857], [50.2383, 170.2039], [50.4839, 172.3546], [50.6878, 174.1401], [51.3, 179.5], [51.6, -176.7], [51.2951, -171.5552], [51.0966, -168.2056], [51.0799, -168.081], [50.7515, -165.623], [50.3734, -162.7942], [50.0, -160.0], [49.1714, -154.5354], [48.9508, -153.4555], [48.5698, -151.5903], [48.2452, -150.0011], [48.0972, -149.2766], [47.6332, -147.484], [46.7971, -144.2543], [46.6354, -143.7413], [45.4566, -140.0027], [45.2935, -139.4855], [44.1117, -136.3287], [43.6057, -134.9769], [43.4858, -134.7002], [42.4942, -132.4113], [42.4421, -132.2909], [40.8893, -130.8343], [40.0, -130.0], [39.4